In [1]:
from spatial_graphs.Alignment import Alignment
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
import os
import pathlib

# Paths

In [2]:
input_path_root = 'C:/Users/mysee/PhD/Code/to_aman/input/'
output_path = 'C:/Users/mysee/PhD/Code/to_aman/output/'

output_path_alignment_global_alignment_centers = output_path + 'Centered/'
output_path_alignment_global_alignment_pcas = output_path + 'PCA_Aligned/'
output_path_alignment_global_alignment_surfaces = output_path + 'Surface_Aligned/'

pathlib.Path(output_path_alignment_global_alignment_centers).mkdir(exist_ok=True)
pathlib.Path(output_path_alignment_global_alignment_pcas).mkdir(exist_ok=True)
pathlib.Path(output_path_alignment_global_alignment_surfaces).mkdir(exist_ok=True)

# Align by pca

In [3]:
for alignment_type in ['Using_Center','Using_PCA','Using_Surface']:
    for hem in ['lhs','rhs']:
        for exp in ['MG49','MG50']:
            print(alignment_type,hem,exp)
            
            if alignment_type == 'Using_Center':
                input_path = input_path_root
                output_path = output_path_alignment_global_alignment_centers
                ref_surf = Surface()
                surf = Surface()

                ref_surf.append(Surface(input_path+\
                                    '{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                ref_surf.append(Surface(input_path+\
                                    '{}_{}_WM_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                
                pia_surf = Surface(input_path+\
                                    '{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf = Surface(input_path+\
                                    '{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                surf.append(pia_surf.surface)
                surf.append(wm_surf.surface)
                
                ref_pts = []
                pts = []
                ref_pts.append(ref_surf.get_center_of_mass())

                pts.append(surf.get_center_of_mass())

                icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
                # write txmat and apply transformation
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.set_transformation_matrix(txmat)
                aligner.write_transformation_matrix(output_path+'{}_{}_centered_txmat.txt'.format(exp,hem))
                surf.apply_icp_transform(icp)
                surf.write_surface_mesh(output_path+'{}_{}_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                pia_surf.apply_icp_transform(icp)
                pia_surf.write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf.apply_icp_transform(icp)
                wm_surf.write_surface_mesh(output_path+'{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                
            elif alignment_type == 'Using_PCA':
                input_path = output_path_alignment_global_alignment_centers
                output_path = output_path_alignment_global_alignment_pcas
                
                ref_pca_pts = []
                pca_pts = []
                ref_surf = Surface()
                surf = Surface()

                ref_surf.append(Surface(input_path_root+\
                                    '{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                ref_surf.append(Surface(input_path_root+\
                                    '{}_{}_WM_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)

                pia_surf = Surface(input_path+\
                                    '{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf = Surface(input_path+\
                                    '{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                surf.append(pia_surf.surface)
                surf.append(wm_surf.surface)

                ref_pca_pts = Alignment().get_pca_for_surf(ref_surf.surface,output_path=None,ref_coronal=True,lhs=(hem=='lhs'))

                pca_pts = Alignment().get_pca_for_surf(surf.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))


                icp,txmat = Landmarks(pts=pca_pts).align_landmarks(ref_pca_pts)
                
                # write txmat and apply transformation
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.set_transformation_matrix(txmat)
                aligner.write_transformation_matrix(output_path+'{}_{}_pca_aligned_txmat.txt'.format(exp,hem))
                surf.apply_icp_transform(icp)
                surf.write_surface_mesh(output_path+'{}_{}_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                pia_surf.apply_icp_transform(icp)
                pia_surf.write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf.apply_icp_transform(icp)
                wm_surf.write_surface_mesh(output_path+'{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))

            else:
                input_path = output_path_alignment_global_alignment_pcas
                output_path = output_path_alignment_global_alignment_surfaces
                #print(input_path,output_path)
                
                ref_surf = Surface()
                surf = Surface()

                ref_surf.append(Surface(input_path_root+\
                                    '/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                #ref_surf.append(Surface(input_path+\
                #                    '/Surfaces/{}_{}_WM_open_bottom_voxel_size_200.vtk'.format('MG48',hem)).surface)

                pia_surf = Surface(input_path+\
                                    '{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf = Surface(input_path+\
                                    '{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                surf.append(pia_surf.surface)
                #surf.append(wm_surf.surface)

                aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                txmat = aligner.get_transformation_matrix()
                icp = aligner.get_icp_transform()
                
                # write txmat and apply transformation
                # write txmat and apply transformation
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.set_transformation_matrix(txmat)
                aligner.write_transformation_matrix(output_path+'{}_{}_surface_aligned_txmat.txt'.format(exp,hem))
                surf.apply_icp_transform(icp)
                surf.write_surface_mesh(output_path+'{}_{}_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                pia_surf.apply_icp_transform(icp)
                pia_surf.write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem))
                wm_surf.apply_icp_transform(icp)
                wm_surf.write_surface_mesh(output_path+'{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem))
            
            #aligner = Alignment(lhs=(hem=='lhs'))
            #aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            #aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)


Using_Center lhs MG49
Using_Center lhs MG50
Using_Center rhs MG49
Using_Center rhs MG50
Using_PCA lhs MG49
Using_PCA lhs MG50
Using_PCA rhs MG49
Using_PCA rhs MG50
Using_Surface lhs MG49
Using_Surface lhs MG50
Using_Surface rhs MG49
Using_Surface rhs MG50
